In [1]:
import requests
#r = requests.post('http://10.30.99.211:8261/gpt_small', data = "What is better for deep learning Python or Matlab?")
#print (r.status_code)

In [3]:
import torch
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='GeForce RTX 2080 Ti', major=7, minor=5, total_memory=11019MB, multi_processor_count=68)

In [4]:
torch.cuda.get_device_properties(0).total_memory

11554717696

In [5]:
torch.cuda.memory_cached(0)

0

In [6]:
torch.cuda.getMemoryUsage(0)

AttributeError: module 'torch.cuda' has no attribute 'getMemoryUsage'

In [19]:
torch.cuda.max_memory_allocated(0)

0

In [7]:
t = torch.cuda.get_device_properties(0).total_memory
c = torch.cuda.memory_cached(0)
a = torch.cuda.memory_allocated(0)
f = c-a  # free inside cache
f

0

In [7]:
for device_index in range(torch.cuda.device_count()):
    print (torch.cuda.get_device_capability(device=device_index))

(7, 5)
(7, 5)
(7, 5)
(7, 5)
(7, 5)
(7, 5)
(7, 5)
(7, 5)


In [18]:
!nvidia-smi

Thu Jun  4 14:07:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 42%   50C    P2    67W / 260W |   7849MiB / 11019MiB |     29%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 43%   52C    P2    78W / 260W |   7945MiB / 11019MiB |     19%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [5]:
torch.cuda.current_device()

4

In [4]:
torch.cuda.set_device(4)

In [11]:
! pwd

/notebook/touche


In [1]:
#numpy
import numpy as np

#torch
import torch, torch.nn as nn
import torch.nn.functional as F

#bert service
from bert_serving.client import BertClient
from sklearn.metrics.pairwise import cosine_similarity
#bc = BertClient()

# transformers
import pytorch_transformers
from pytorch_transformers import *



In [ ]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [2]:
from xml.dom import minidom
import re

def read_xml(filename):
    # convert file filename to list of tuples (number_of_topic, title_of_topic) 
    # input: filename string
    # output: list of corresponding tuples
    answer_list = []
    xmldoc = minidom.parse(filename)
    itemlist = xmldoc.getElementsByTagName('topics')
    print(len(itemlist))
    print(itemlist)
    topic_list = itemlist[0].getElementsByTagName('topic')
    print (len(topic_list))
    for topic in topic_list:
        tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
        answer_list.append(tuple_for_add)
    return answer_list

def make_a_search_request(query, size=10):
    # return json
    # json will be processed further
    params = {
            "apikey": "0833a307-97d3-462a-99d9-27db400c70da",
            "query": query,
            "index": ["cw12"],
            "size": size,
            "pretty": True
        }
    response = requests.get(url = "http://www.chatnoir.eu/api/v1/_search", params = params)
    return response

def clean_punct(s):
    s = re.sub(r'[^\w\s]','',s)
    return s

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()


In [3]:
def make_scores_1(query, answer_titles):
    query_emb = bc.encode([query])
    scores = [cosine_similarity(query_emb.reshape(1, -1), bc.encode([cleanhtml(answer_title)]).reshape(1, -1))[0][0] for answer_title in answer_titles]
    return scores

In [ ]:
def make_scores_transformers(model, query, answers_bodies):
    scores = []
    for ind, elem in enumerate(answers_bodies):
        sentences = ["[CLS] " + query + " [SEP] " + answers_bodies[ind] + " [SEP]"]
        #print(sentences[0])

        # Tokenize with BERT tokenizer
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
        #print ("Tokenize the first sentence:")
        #print (tokenized_texts[0])
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
        # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
        segments_ids = (len(tokenizer.tokenize(query)) + 2)*[0] + (len(tokenizer.tokenize(answers_bodies[ind])) + 1)*[1]
        cls_idx = 0
        sep_idx = len(tokenizer.tokenize(query))
        end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answers_bodies[ind])) + 3
        #print (len(tokenizer.tokenize(query)), len(tokenizer.tokenize(answers_bodies[ind])))
        #print (len(indexed_tokens), len(segments_ids))
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        answ = model(tokens_tensor, segments_tensors)

        attensions = answ[2][3][0][0].detach().numpy() + answ[2][9][0][0].detach().numpy()
        #probs[cls_idx:sep_idx, sep_idx:] = 0.0
        score = np.sum(attensions[cls_idx:sep_idx, sep_idx:]) + np.sum(attensions[sep_idx:end_idx, cls_idx:sep_idx+2])
        #print (score)
        scores.append(score)
    return scores
    

In [3]:
my_response_list = load_obj("list_of_un_answ")

In [4]:
my_response_list['3'][:20]

[(2317.332,
  'clueweb12-0200wb-72-02633',
  'canon eos 600d vs. nikon d3100 - which is better?',
  'canon eos 600d with 18-55mm f&#x2f;3.5-5.6 is ii lens kit sign in to post a comment | you can also sign in with all rights reserved.'),
 (1792.2296,
  'clueweb12-0200wb-72-02639',
  'nikon d5100 vs canon eos 600d (rebel t3i) - which one is better?',
  'if you cannot locate the when you search for friend via google, gmail, yahoo or hotmail, we have no access to your login and password. a pop up window will appear which is hosted by google, yahoo or microsoft, and a list of email addresses are passed to us via secure connection.'),
 (1312.001,
  'clueweb12-0400tw-46-07931',
  'nikon d800 vs. nikon d4 - which one to buy?',
  'if you cannot locate the when you search for friend via google, gmail, yahoo or hotmail, we have no access to your login and password. a pop up window will appear which is hosted by google, yahoo or microsoft, and a list of email addresses are passed to us via secure 

In [17]:
def run_baseline(output_dir = '/notebook/touche/output/', input_dir = '/notebook/touche/', input_file = 'topics-task-2-only-titles.xml'):
    
    #transformer part
    config_class, model_class, tokenizer_class = BertConfig, BertModel, BertTokenizer
    config = config_class.from_pretrained('bert-base-uncased')
    config.output_attentions=True
    model = model_class.from_pretrained('bert-base-uncased', config=config)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    
    # load responses for all queries from file 
    my_response_list = load_obj("list_of_un_answ")
    
    list_of_tuples = read_xml(input_dir + input_file)
    common_list = []
    
        
    print ("attention", flush = True)    
    with open(output_dir + 'transformer.txt', 'w') as trnsfm:
        for ind_q, elem in enumerate(list_of_tuples[:5]):
            qid = elem[0]
            Q0 = 'Q0'
            query = elem[1]
            tag = 'MethodAttentionFilterResponse'
            responses = list(zip(*my_response_list[str(ind_q + 1)]))

            scores0 = responses[0]
            print ("0")
            docs = responses[1]
            print ("1")
            titles = responses[2]
            #print ("2")
            answers_bodies = responses[3]
            #print ("3")
            # print (scores0, scores3, scores)
            scores = make_scores_transformers(model, query, answers_bodies)
            qids = qid*len(scores)
            Q0s = [Q0 for elem in scores]
            queries = query*len(scores)
            tags = [tag for elem in scores]
            
            help_list = list(zip(qids, Q0s, queries, scores, titles, answers_bodies))
            help_list = sorted(help_list, key = lambda x: x[3], reverse = True)
            trnsfm.write('\n'.join('%s %s %s %s %s %s' % x for x in help_list))
            trnsfm.write('\n')
            
            part_of_commom_list = list(zip(qids, Q0s, docs, scores, tags))
            part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 
            

            qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)

            ranks = range(1, len(scores) + 1)
            part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
            common_list = common_list + part_of_commom_list
        
    print ("baseline", flush = True)
    with open(output_dir + 'baseline.txt', 'w') as bsln:
        for ind_q, elem in enumerate(list_of_tuples[:3]):
            qid = elem[0]
            Q0 = 'Q0'
            query = elem[1]
            tag = 'MyBaselineFilterResponse'
            responses = list(zip(*my_response_list[str(ind_q + 1)]))

            scores0 = responses[0]
            #print ("0")
            docs = responses[1]
            #print ("1")
            titles = responses[2]
            #print ("2")
            answers_bodies = responses[3]
            #print ("3")
            # print (scores0, scores3, scores)
            qids = qid*len(scores)
            Q0s = [Q0 for elem in scores]
            tags = [tag for elem in scores]
            part_of_commom_list = list(zip(qids, Q0s, docs, scores0, tags))
            part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 

            qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)
            
            help_list = list(zip(qids, Q0s, queries, scores, titles, answers_bodies))
            help_list = sorted(help_list, key = lambda x: x[3], reverse = True)
            bsln.write('\n'.join('%s %s %s %s %s %s' % x for x in help_list))
            bsln.write('\n')

            ranks = range(1, len(scores) + 1)
            part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
            common_list = common_list + part_of_commom_list
            part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
            common_list = common_list + part_of_commom_list
        
    print ("bert embeddings", flush = True)
    with open(output_dir + 'bert_e.txt', 'w') as bert_e:
        for ind_q, elem in enumerate(list_of_tuples[:5]):
            qid = elem[0]
            Q0 = 'Q0'
            query = elem[1]
            tag = 'myBertSimilarity'
            responses = list(zip(*my_response_list[str(ind_q + 1)]))

            scores0 = responses[0]
            #print ("0")
            docs = responses[1]
            #print ("1")
            titles = responses[2]
            #print ("2")
            answers_bodies = responses[3]
            #print ("3")
            # print (scores0, scores3, scores)
            scores = make_scores_1(query, titles)
            qids = qid*len(scores)
            Q0s = [Q0 for elem in scores]
            tags = [tag for elem in scores]
            part_of_commom_list = list(zip(qids, Q0s, docs, scores0, tags))
            part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 

            qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)

            help_list = list(zip(qids, Q0s, queries, scores, titles, answers_bodies))
            help_list = sorted(help_list, key = lambda x: x[3], reverse = True)
            bert_e.write('\n'.join('%s %s %s %s %s %s' % x for x in help_list))
            bert_e.write('\n')

            ranks = range(1, len(scores) + 1)
            part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
            common_list = common_list + part_of_commom_list
            part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
            common_list = common_list + part_of_commom_list

    with open(output_dir + 'run_example5_22_05.txt', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

In [9]:
run_baseline()

1
[<DOM Element: topics at 0x7f3913ccdd70>]
50
attention
0
1
0
1
0
1
0
1
0
1
baseline
bert embeddings


In [4]:
config_class, model_class, tokenizer_class = BertConfig, BertModel, BertTokenizer
config = config_class.from_pretrained('bert-base-uncased')
config.output_attentions=True
model = model_class.from_pretrained('bert-base-uncased', config=config)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [10]:
answ[0].shape

NameError: name 'answ' is not defined

In [11]:
answ[1].shape

NameError: name 'answ' is not defined

In [12]:
answ[2][3].shape

NameError: name 'answ' is not defined

In [5]:
import bokeh.plotting as pl
import bokeh.models as bm
from bokeh.io import output_notebook, show
output_notebook()

def draw_attention(inp_tokens, trans_tokens, probs):
    """ An intentionally ambiguous function to visualize attention weights """
    probs = probs[:len(trans_tokens), :len(inp_tokens)]
    
    fig = pl.figure(x_range=(0, len(inp_tokens)), y_range=(0, len(trans_tokens)),
                    x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs[::-1]], 0, 0, len(inp_tokens), len(trans_tokens))

    fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
    fig.xaxis.ticker = np.arange(len(inp_tokens)) + 0.5
    fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens)) + 0.5, inp_tokens))
    fig.xaxis.major_label_orientation = 45

    fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
    fig.yaxis.ticker = np.arange(len(trans_tokens)) + 0.5
    fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens)) + 0.5, trans_tokens[::-1]))

    show(fig)

Loading BokehJS ...

In [390]:
np.arange(len(inp_tokens)) + 0.5

array([0.5])

Идея - из аттеншн слоев выбрать те, которые в левом вернем и правом нижнем углу (0 предложение на себя и  предложение на себя) не дают полностью черных квадратов. 3 слой - атеншн на аналогичные слова

обнулить sep-tokena

Просуммировать левый верхний и правый нижний квадрат - для каждого поискового ответа

In [391]:
probs = answ[2][ii][0][0].detach().numpy()

probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]

In [392]:
probs

array([[2.8974013e-02, 7.1487352e-03, 8.6870305e-03, ..., 1.4310145e-04,
        3.3603467e-02, 2.2732976e-01],
       [9.8076705e-03, 5.7020295e-02, 2.0104237e-02, ..., 3.5717709e-05,
        4.3609999e-02, 1.9834188e-01],
       [5.1948074e-03, 2.1921040e-02, 6.8340912e-02, ..., 7.3892690e-05,
        1.0941007e-02, 8.5741229e-02],
       ...,
       [3.8002472e-04, 8.3944714e-04, 3.2824180e-05, ..., 1.9493585e-02,
        9.4970554e-02, 1.5036830e-01],
       [1.0896232e-03, 8.6444616e-04, 6.8232120e-04, ..., 3.3967728e-05,
        7.4814504e-04, 1.7166950e-01],
       [1.8940828e-03, 1.9991521e-03, 1.2598590e-03, ..., 8.3928338e-05,
        7.1911293e-04, 1.7222546e-01]], dtype=float32)

In [393]:
cls_idx = 0
sep_idx = len(tokenizer.tokenize(query))
end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answers_bodies[10])) + 3

In [394]:
probs = answ[2][ii][0][0].detach().numpy()
probs = probs[cls_idx:sep_idx, sep_idx:end_idx]
probs.shape

(9, 43)

In [395]:
len(trans_tokens[0][cls_idx:sep_idx])

9

In [6]:
query = 'Which is better, a laptop or a desktop?'
answer1 = 'Laptop is preferable than desktop because it is more portable.'
answer2 = 'If you need portability, the laptop is the best option than desktop.'
answer3 = 'The crew was complete: it included a boots, a maker of bonnets and hoods'
answer4 = 'Just the pace for snark the Bellman cried'

Expected results: answer1 and answer2 are better than answer3

In [28]:
from bokeh.layouts import row, column
from bokeh.io import export_png
def show_importance_of_transformers_layer(query, answer):

    # return scores - list of len = number_of_transformers_heads
    # every element is sum of weight in left top and rinht bottom square - attention weights which correspond to tokens 
    # not in word's sentence. 
    # To estimate closeness between sentences A and B for every word in A we count all weight corresponding to word in B

    sentences = ["[CLS] " + query + " [SEP] " + answer + " [SEP]"]
    print(sentences[0])

    # Tokenize with BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    print ("Tokenize the first sentence:")
    print (tokenized_texts[0])
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
    # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
    segments_ids = (len(tokenizer.tokenize(query)) + 2)*[0] + (len(tokenizer.tokenize(answer)) + 1)*[1]
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    answ = model(tokens_tensor, segments_tensors)
    trans_tokens = tokenized_texts
    inp_tokens = tokenized_texts
    cls_idx = 0
    sep_idx = len(tokenizer.tokenize(query))
    end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answer)) + 3

    print (cls_idx, sep_idx, end_idx)
    scores = []
    list_of_figures = []
    for ii in range(12):
        probs = answ[2][ii][0][0].detach().numpy()

        probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]
        score = np.sum(probs[cls_idx+1:sep_idx-1, sep_idx+1:end_idx -1]) + np.sum(probs[sep_idx+1:end_idx-1, cls_idx+1:sep_idx-2])
        fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                        x_axis_type=None, y_axis_type=None, tools=[], plot_width=240, plot_height=240, title = 'head ' + str(ii))
        fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))
        #export_png(fig, filename="trans_picturs\plot" + str(ii) +".png")
        scores.append(score)
        list_of_figures.append(fig)
        #fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
        #fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
        #fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
        #fig.xaxis.major_label_orientation = 45

        #fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
        #fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
        #fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))

    show(row(list_of_figures[:4]))
    show(row(list_of_figures[4:8]))
    show(row(list_of_figures[8:12]))
    return scores

**Что берем?**

**3 слой** - я ищет похожие слова, 

**4, 10, 11** - наибольший divergence (см. scores)

In [27]:
print (scores_1)
print (scores_2)
print (scores_3)
print (scores_4)

[1.6208771, 3.7034912, 0.0006053873, 5.5407987, 6.797483, 2.0428846, 4.585038, 1.3815542, 3.465903, 6.005816, 6.0510125, 4.3819027]
[1.7041466, 4.2792974, 0.0007035341, 5.0892878, 7.537561, 1.9327469, 4.6909723, 1.2563944, 3.6153867, 6.3739057, 6.816018, 5.550683]
[1.7991419, 4.191726, 0.0011484785, 4.8690877, 4.970048, 1.7203952, 3.3389707, 1.351007, 3.0996885, 4.975832, 3.7327018, 1.107487]
[1.6725659, 3.4933279, 0.00052020757, 2.8354619, 4.331377, 1.9027277, 3.3027313, 1.4085783, 3.4222302, 4.476509, 3.9202418, 3.0011702]


In [29]:
scores_1 = show_importance_of_transformers_layer(query, answer1)

[CLS] Which is better, a laptop or a desktop? [SEP] Laptop is preferable than desktop because it is more portable. [SEP]
Tokenize the first sentence:
['[CLS]', 'which', 'is', 'better', ',', 'a', 'laptop', 'or', 'a', 'desktop', '?', '[SEP]', 'laptop', 'is', 'prefer', '##able', 'than', 'desktop', 'because', 'it', 'is', 'more', 'portable', '.', '[SEP]']
0 10 25


In [30]:
scores_2 = show_importance_of_transformers_layer(query, answer2)

[CLS] Which is better, a laptop or a desktop? [SEP] If you need portability, the laptop is the best option than desktop. [SEP]
Tokenize the first sentence:
['[CLS]', 'which', 'is', 'better', ',', 'a', 'laptop', 'or', 'a', 'desktop', '?', '[SEP]', 'if', 'you', 'need', 'port', '##ability', ',', 'the', 'laptop', 'is', 'the', 'best', 'option', 'than', 'desktop', '.', '[SEP]']
0 10 28


In [31]:
scores_3 = show_importance_of_transformers_layer(query, answer3)

[CLS] Which is better, a laptop or a desktop? [SEP] The crew was complete: it included a boots, a maker of bonnets and hoods [SEP]
Tokenize the first sentence:
['[CLS]', 'which', 'is', 'better', ',', 'a', 'laptop', 'or', 'a', 'desktop', '?', '[SEP]', 'the', 'crew', 'was', 'complete', ':', 'it', 'included', 'a', 'boots', ',', 'a', 'maker', 'of', 'bonnet', '##s', 'and', 'hood', '##s', '[SEP]']
0 10 31


In [26]:
scores_4 = show_importance_of_transformers_layer(query, answer4)

[CLS] Which is better, a laptop or a desktop? [SEP] Just the pace for snark the Bellman cried [SEP]
Tokenize the first sentence:
['[CLS]', 'which', 'is', 'better', ',', 'a', 'laptop', 'or', 'a', 'desktop', '?', '[SEP]', 'just', 'the', 'pace', 'for', 's', '##nar', '##k', 'the', 'bell', '##man', 'cried', '[SEP]']
0 10 24


In [41]:
sentences = ["[CLS] " + query + " [SEP] " + answer1 + " [SEP]"]
print(sentences[0])

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = (len(tokenizer.tokenize(query)) + 2)*[0] + (len(tokenizer.tokenize(answer1)) + 1)*[1]
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
answ = model(tokens_tensor, segments_tensors)
trans_tokens = tokenized_texts
inp_tokens = tokenized_texts
cls_idx = 0
sep_idx = len(tokenizer.tokenize(query))
end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answer1)) + 3

print (cls_idx, sep_idx, end_idx)
for ii in [3]:
    probs = answ[2][ii][0][0].detach().numpy()

    probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]

    fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                    x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))
    fig.add_layout(bm.LinearAxis(axis_label=''), 'above')
    fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
    fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
    fig.xaxis.major_label_text_font_size = "11pt"
    fig.xaxis.major_label_orientation = 45

    fig.add_layout(bm.LinearAxis(axis_label=''), 'left')
    fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
    fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))
    fig.yaxis.major_label_text_font_size = "11pt"
    #fig.export_png("3rdlayer.png")
    show(fig)

[CLS] Which is better, a laptop or a desktop? [SEP] Laptop is preferable than desktop because it is more portable. [SEP]
Tokenize the first sentence:
['[CLS]', 'which', 'is', 'better', ',', 'a', 'laptop', 'or', 'a', 'desktop', '?', '[SEP]', 'laptop', 'is', 'prefer', '##able', 'than', 'desktop', 'because', 'it', 'is', 'more', 'portable', '.', '[SEP]']
0 10 25


In [33]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
answ = model(tokens_tensor, segments_tensors)

NameError: name 'indexed_tokens' is not defined

In [28]:
trans_tokens = tokenized_texts
inp_tokens = tokenized_texts
cls_idx = 0
sep_idx = len(tokenizer.tokenize(query))
end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answer1)) + 3

print (cls_idx, sep_idx, end_idx)

for ii in range(12):
    probs = answ[2][ii][0][0].detach().numpy()

    probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]
    #probs[cls_idx:sep_idx, sep_idx:] = 0.0
    #probs[sep_idx:end_idx, cls_idx:sep_idx+2] = 0.0
    
    #nullify sep - need we or not?
    #probs1 = np.copy(probs)
    #probs1[0] = 0.0
    #probs1[:, 0] = 0.0
    #probs1[10] = 0.0
    #probs1[len(probs)-1] = 0.0
    #probs1[:, 10] = 0.0
    #probs1[:, len(probs)-1] = 0.0

    fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                    x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))
    score = np.sum(probs[cls_idx:sep_idx, sep_idx+1:]) + np.sum(probs[sep_idx+1:end_idx, cls_idx:sep_idx+2])
    print (score, flush = True)
    #fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
    #fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
    #fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
    #fig.xaxis.major_label_orientation = 45

    #fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
    #fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
    #fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))

    show(fig)

0 10 30
3.923529


14.189812


1.4350145


19.661015


17.780321


...

11.441083


16.026836


16.497343


14.942686


14.82329


In [32]:
trans_tokens = tokenized_texts
inp_tokens = tokenized_texts

for ii in [3, 9]:
    probs = answ[2][ii][0][0].detach().numpy()

    probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]
    probs = probs[cls_idx:sep_idx, :]
    
    #nullify sep - need we or not?
    #probs1 = np.copy(probs)
    #probs1[0] = 0.0
    #probs1[:, 0] = 0.0
    #probs1[10] = 0.0
    #probs1[len(probs)-1] = 0.0
    #probs1[:, 10] = 0.0
    #probs1[:, len(probs)-1] = 0.0

    fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                    x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))

    fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
    fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
    fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
    fig.xaxis.major_label_orientation = 45

    fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
    fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
    fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))

    show(fig)

NameError: name 'tokenized_texts' is not defined

In [203]:
len(segments_ids)

21

In [19]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
answ = model(tokens_tensor, segments_tensors)

NameError: name 'model' is not defined

In [61]:
trans_tokens = tokenized_texts
inp_tokens = tokenized_texts
for ii in range(11):
    probs = answ[2][ii][0][0].detach().numpy()
    probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]
    fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                        x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))

    fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
    fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
    fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
    fig.xaxis.major_label_orientation = 45

    fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
    fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
    fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))

    show(fig)

In [59]:
sentences = ["[CLS] " + query + " [SEP] " + 'Just the place for a Snark!" the Bellman cried' + " [SEP]"]
print(sentences[0])

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] + 15*[1]

tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
answ = model(tokens_tensor, segments_tensors)

trans_tokens = tokenized_texts
inp_tokens = tokenized_texts

probs = answ[2][9][0][0].detach().numpy()
probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]
fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                    x_axis_type=None, y_axis_type=None, tools=[])
fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))

fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
fig.xaxis.major_label_orientation = 45

fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))

show(fig)

[CLS] Which is better, a laptop or a desktop? [SEP] Just the place for a Snark!" the Bellman cried [SEP]
Tokenize the first sentence:
['[CLS]', 'which', 'is', 'better', ',', 'a', 'laptop', 'or', 'a', 'desktop', '?', '[SEP]', 'just', 'the', 'place', 'for', 'a', 's', '##nar', '##k', '!', '"', 'the', 'bell', '##man', 'cried', '[SEP]']


RuntimeError: The size of tensor a (27) must match the size of tensor b (26) at non-singleton dimension 1

In [17]:
def run_baseline(output_dir = '/notebook/touche/output/', input_dir = '/notebook/touche/', input_file = 'topics-task-2-only-titles.xml'):
    list_of_tuples = read_xml(input_dir + input_file)
    common_list = []
    
    with open(output_dir + 'run_example.txt', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
        
    print ("lstm")    
    for elem in list_of_tuples:
        qid = elem[0]
        Q0 = 'Q0'
        query = elem[1]
        tag = 'Method3'
        response = make_a_search_request(query)
        try:
            getted_request = response.json()
        except:
            print ("exept0")
            return getted_request
        
        n_request = 100
        while (n_request > 0 and 'results' not in getted_request):
            response = make_a_search_request(query)
            getted_request = response.json()
            n_request == n_request - 1
            print ("rerequest", query)
            
        try:
            scores0 = [elem['score'] for elem in getted_request['results']]
            print ("0")
            docs = [elem['trec_id'] for elem in getted_request['results']]
            print ("1")
            titles = [elem['title'] for elem in getted_request['results']]
            print ("2")
            answers_bodies = [cleanhtml(elem['snippet']) for elem in getted_request['results']]
            print ("3")
            # print (scores0, scores3, scores)
        except:
            print ("exept1")
            return getted_request
        scores = make_scores_lstm(query, answers_bodies)
        qids = qid*len(scores)
        Q0s = [Q0 for elem in scores]
        queries = query*len(scores)
        tags = [tag for elem in scores]
        part_of_commom_list = list(zip(qids, Q0s, docs, scores, tags))
        part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 
        
        qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)
        
        ranks = range(1, len(scores) + 1)
        part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
        common_list = common_list + part_of_commom_list

    with open(output_dir + 'run_example1.txt', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

In [ ]:
run_baseline()

1
[<DOM Element: topics at 0x7f327e9e1b90>]
50
lstm
0
1
2
3
aa shape (1, 1, 64)
bb shape (1, 1, 64)
0
1
2
3
aa shape (1, 1, 64)
bb shape (1, 1, 64)
rerequest Which is better, Canon or Nikon?
0
1
2
3
aa shape (1, 1, 64)
bb shape (1, 1, 64)
0
1
2
3
aa shape (1, 1, 64)
bb shape (1, 1, 64)
0
1
2
3
aa shape (1, 1, 64)
bb shape (1, 1, 64)
0
1
2
3
